**MODELO CONCEITUAL (Versao 1.3)**

![title](../img/modelo-conceitual-1.3.png)

__MODELO RELACIONAL__

Doenca(__id__, nome, categoria)

Paciente(__id__, tipo, altura, peso, sexo, idade)

Sintoma(__codigo__, nome)

PacienteSintoma(__idPaciente__, __codSintoma__)<br/>
FK: idPaciente - Paciente(id)<br/>
FK: codSintoma - Sintoma(codigo)

PacienteHistoricoDoencas(__idPaciente__, __idDoenca__)<br/>
FK: idPaciente - Paciente(id)<br/>
FK: idDoenca - Doenca(id)

ContaMedico(__username__, senha, nome, sobrenome, email, numCRM, cidadeFederativa, situacao)

Remedio(__id__, principioAtivo, laboratorio, nomeVenda, apresentacao, classeTerapeutica, tipo, restricaoHospitalar, tarja)

Tratamento(__id__, __idRemedio__, apreDosagem, dosis_mcg, dosis_mg, dosis_ml, numGotas, repeticao, numVezesDia, numDias)<br/>
FK: idRemedio - Remedio(id)

Recomendacao(__codigo__, usernameMedico, idPaciente, __idTratamento__)<br/>
FK: usernameMedico - Medico(username)<br/>
FK: idPaciente - Paciente(id)

__SQL__

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Doenca;
DROP TABLE IF EXISTS Paciente;
DROP TABLE IF EXISTS Sintoma;
DROP TABLE IF EXISTS PacienteSintoma;
DROP TABLE IF EXISTS PacienteHistoricoDoencas;
DROP TABLE IF EXISTS ContaMedico;
DROP TABLE IF EXISTS Remedio;
DROP TABLE IF EXISTS Tratamento;
DROP TABLE IF EXISTS Recomendacao;

CREATE TABLE Doenca (
  id VARCHAR(5) NOT NULL,
  nome VARCHAR(50) NOT NULL,
  categoria VARCHAR(50) NULL,
  PRIMARY KEY(id)
) AS SELECT
    id,
    nome,
    categoria
FROM CSVREAD('../data/Doenca.csv');

CREATE TABLE Paciente (
  id VARCHAR(10) NOT NULL,
  tipo CHAR NOT NULL,
  altura DOUBLE NOT NULL,
  peso DOUBLE NOT NULL,
  sexo CHAR NOT NULL,
  idade INTEGER NOT NULL,
  PRIMARY KEY(id)
) AS SELECT
    id,
    tipo,
    altura,
    peso,
    sexo,
    idade
FROM CSVREAD('../data/Paciente.csv');

CREATE TABLE Sintoma (
  codigo VARCHAR(5) NOT NULL,
  nome VARCHAR(50) NOT NULL,
  PRIMARY KEY(codigo)
) AS SELECT
    codigo,
    nome
FROM CSVREAD('../data/Sintoma.csv');

CREATE TABLE PacienteSintoma (
  idPaciente VARCHAR(10) NOT NULL,
  codSintoma VARCHAR(5) NOT NULL,
  PRIMARY KEY(idPaciente, codSintoma),
  FOREIGN KEY(idPaciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codSintoma)
    REFERENCES Sintoma(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    idPaciente,
    codSintoma
FROM CSVREAD('../data/PacienteSintoma.csv');

CREATE TABLE PacienteHistoricoDoencas (
  idPaciente VARCHAR(10) NOT NULL,
  idDoenca VARCHAR(5) NOT NULL,
  PRIMARY KEY(idDoenca, idPaciente),
  FOREIGN KEY(idPaciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(idDoenca)
    REFERENCES Doenca(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    idPaciente,
    idDoenca
FROM CSVREAD('../data/PacienteHistoricoDoencas.csv');

CREATE TABLE ContaMedico (
  username VARCHAR(20) NOT NULL,
  senha VARCHAR(20) NOT NULL,
  nome VARCHAR(50) NOT NULL,
  sobrenome VARCHAR(50) NOT NULL,
  email VARCHAR(70) NOT NULL,
  numCRM INTEGER NOT NULL,
  cidadeFederativa VARCHAR(3) NOT NULL,
  situacao CHAR NOT NULL,
  PRIMARY KEY(username)
) AS SELECT
    username,
    senha,
    nome,
    sobrenome,
    email,
    numCRM,
    cidadeFederativa,
    situacao
FROM CSVREAD('../data/ContaMedico.csv');

CREATE TABLE Remedio (
  id VARCHAR(20) NOT NULL,
  principioAtivo VARCHAR(250) NOT NULL,
  laboratorio VARCHAR(150) NOT NULL,
  nomeVenda VARCHAR(150) NOT NULL,
  apresentacao VARCHAR(200) NULL,
  classeTerapeutica VARCHAR(150) NULL,
  tipo VARCHAR(150) NOT NULL,
  restricaoHospitalar VARCHAR(50) NULL,
  tarja VARCHAR(15) NULL,
  PRIMARY KEY(id)
) AS SELECT
    ID_REMEDIO,
    PRINCIPIO_ATIVO,
    LABORATORIO,
    PRODUTO,
    APRESENTACAO,
    CLASSE_TERAPEUTICA,
    TIPO_DE_PRODUTO,
    RESTRICAO_HOSPITALAR,
    TARJA
FROM CSVREAD('../data/Remedio.csv');

CREATE TABLE Tratamento (
  id VARCHAR(20) NOT NULL,
  idRemedio VARCHAR(20) NOT NULL,
  apreDosagem VARCHAR(20) NOT NULL,
  dosis_mcg DOUBLE NULL,
  dosis_mg DOUBLE NULL,
  dosis_ml DOUBLE NULL,
  numGotas DOUBLE NULL,
  repeticao INTEGER NULL,
  numVezesDia INTEGER NULL,
  numDias INTEGER NULL,
  PRIMARY KEY(id, idRemedio),
  FOREIGN KEY(idRemedio)
    REFERENCES Remedio(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    id,
    idRemedio,
    apreDosagem,
    dosis_mcg,
    dosis_mg,
    dosis_ml,
    numGotas,
    repeticao,
    numVezesDia,
    numDias
FROM CSVREAD('../data/Tratamento.csv');

CREATE TABLE Recomendacao (
  codigo VARCHAR(10) NOT NULL,
  usernameMedico VARCHAR(20) NOT NULL,
  idPaciente VARCHAR(10) NOT NULL,
  idTratamento VARCHAR(20) NOT NULL,
  PRIMARY KEY(codigo, idTratamento),
  FOREIGN KEY(usernameMedico)
    REFERENCES ContaMedico(username)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(idPaciente)
    REFERENCES Paciente(id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
    --fazendo referencia à tabela Tratamento se apresenta o erro: 'Primary key violation',
    --devido a que só se quere fazer referencia a um elemento (id) da superchave (id, idRemedio)
) AS SELECT
    codigo,
    usernameMedico,
    idPaciente,
    idTratamento
FROM CSVREAD('../data/Recomendacao.csv');

**CONSULTAS - SQL**

1) Quais sao os medicamentos escolhidos pelos doutores para tratar sintomas como: lacrimejo e coceira-olhos.<br/>
Obs: ter em conta que estos sintomas se podem apresentar juntos com outros.

In [3]:
--primeiro obtemos os codigos de esos sintomas
--segundo obtemos os pacientes que apresentam estos sintomas (estos pacientes podem ter mais sintomas)
--terceiro se obtem os tratamento para esos pacientes (mas estos tratamentos podem ser para outros sintomas)
--finalmente se obtem os remedios para os tratamentos
SELECT R.idPaciente, T1.numSintomas, R.idTratamento, Tra.idRemedio, Re.nomeVenda, Re.apresentacao FROM Recomendacao R, Tratamento Tra, Remedio Re
INNER JOIN (SELECT PS1.idPaciente, COUNT(DISTINCT PS2.codSintoma) AS numSintomas FROM PacienteSintoma PS1, PacienteSintoma PS2
WHERE PS1.codSintoma IN (SELECT S.codigo FROM Sintoma AS S
                    WHERE S.nome = 'lacrimejo' OR S.nome = 'coceira-olhos')
AND PS2.idPaciente = PS1.idPaciente
GROUP BY PS2.idPaciente
ORDER BY numSintomas) T1
ON R.idPaciente = T1.idPaciente
WHERE R.idTratamento = Tra.id
AND Tra.idRemedio = Re.id

2) Que tipo de remedios sao receitados às pessoas que sofrem de pressao alta, quando apresentan congestionamento nasal.

In [4]:
--primeiro se obtem o codigo do sintoma solicitado
--segundo se obtem os pacientes que sofrem de pressao alta e tem o sintoma
--depois se obtem os medicamentos indicados pelos doutores em seus tratamentos
SELECT R.idPaciente, R.idTratamento, Tra.idRemedio, Re.nomeVenda, Re.apresentacao FROM Recomendacao R, Tratamento Tra, Remedio Re
WHERE R.idPaciente IN (SELECT PHD.idPaciente FROM PacienteHistoricoDoencas PHD, Doenca D
                    INNER JOIN (SELECT DISTINCT PS.idPaciente FROM PacienteSintoma PS
                    WHERE PS.codSintoma IN (SELECT S.codigo FROM Sintoma AS S
                                        WHERE S.nome = 'congestao nasal')) T1
                    ON PHD.idPaciente = T1.idPaciente
                    AND PHD.idDoenca = D.id
                    WHERE D.nome = 'Pressao alta')
AND R.idTratamento = Tra.id
AND Tra.idRemedio = Re.id
ORDER BY R.idPaciente

3) Quando os pacientes que sofrem de insuficiencia renal tem sintomas de alergia (lacrimejo, coceira-olhos, coriza ou vermelhidao), a quantidade de medicamento receitado segue algum padrao?

In [9]:
--se obtem as caracteristicas dos tratamentos indicados aos pacientes que sofrem de insuficiencia renal e tem sintomas de alergia
SELECT R.idPaciente, Tra.*, Re.nomeVenda, T.nomeDoenca FROM Recomendacao R, Tratamento Tra, Remedio Re
INNER JOIN
(SELECT PHD.idPaciente, PHD.idDoenca, D.nome AS nomeDoenca FROM PacienteHistoricoDoencas PHD, Doenca D
INNER JOIN (SELECT DISTINCT PS.idPaciente FROM PacienteSintoma PS
WHERE PS.codSintoma IN (SELECT S.codigo FROM Sintoma AS S
                    WHERE S.nome = 'lacrimejo' OR S.nome = 'coceira-olhos' OR S.nome = 'coriza' OR S.nome = 'vermelhidao')) T1
ON PHD.idPaciente = T1.idPaciente
AND PHD.idDoenca = D.id
WHERE nome = 'Insuficiência renal normal' OR nome = 'Insuficiencia renal media' OR nome = 'Insuficiencia renal moderada' OR nome = 'Insuficiencia renal grave') T
ON R.idPaciente = T.idPaciente
AND R.idTratamento = Tra.id
AND Tra.idRemedio = Re.id
ORDER BY Tra.dosis_mg
--se observa que a dosis é menor a um maior nivel de enfermedad

4) Quando uma crianca tem tosse, os remedios receitados seguem algum padrao relacionado com o peso?

In [25]:
--primero se obtem as criancas que sofrem de tosse
--segundo se os remedios receitados para eles
SELECT R.idPaciente, T.peso, Tra.*, Re.nomeVenda, Re.apresentacao FROM Recomendacao R, Tratamento Tra, Remedio Re
INNER JOIN
(SELECT PA.* FROM Paciente PA
WHERE PA.tipo = 'C'
AND PA.id IN (SELECT DISTINCT PS.idPaciente FROM PacienteSintoma PS
            WHERE PS.codSintoma IN (SELECT S.codigo FROM Sintoma AS S
                                    WHERE S.nome = 'tosse'))) T
ON R.idPaciente = T.id
AND R.idTratamento = Tra.id
AND Tra.idRemedio = Re.id
ORDER BY R.idPaciente
--se observa que o numero de gotas coincide com seu peso

**CONSULTAS - CYPHER**

---Criacao de dados---
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/obi10/trabalhoDados/master/jupyter/data/Paciente.csv' AS line
CREATE (:Paciente { id: line.id, tipo: line.tipo, altura: line.altura, peso: line.peso, sexo: line.sexo, idade: line.idade})

CREATE INDEX ON :Paciente(id)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/obi10/trabalhoDados/master/jupyter/data/Sintoma.csv' AS line
CREATE (:Sintoma { codigo: line.codigo, nome: line.nome})

CREATE INDEX ON :Sintoma(codigo)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/obi10/trabalhoDados/master/jupyter/data/PacienteSintoma.csv' AS line
MATCH (p:Paciente {id: line.idPaciente})
MATCH (s:Sintoma {codigo: line.codSintoma})
CREATE (p)-[:tem]->(s)

MATCH (p)-[:tem]->(s)
RETURN p, s

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/obi10/trabalhoDados/master/jupyter/data/Doenca.csv' AS line
CREATE (:Doenca { id: line.id, nome: line.nome, categoria: line.categoria})

CREATE INDEX ON :Doenca(id)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/obi10/trabalhoDados/master/jupyter/data/PacienteHistoricoDoencas.csv' AS line
MATCH (p:Paciente {id: line.idPaciente})
MATCH (d:Doenca {id: line.idDoenca})
CREATE (p)-[:historico]->(d)

MATCH (p)-[:tem]->(s)
MATCH (p)-[:historico]->(d)
RETURN p, s, d

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/obi10/trabalhoDados/master/jupyter/data/ContaMedico.csv' AS line
CREATE (:Medico { username: line.username, senha: line.senha, nome: line.nome, sobrenome: line.sobrenome, email: line.email, numCRM: line.numCRM, cidadeFederativa: line.cidadeFederativa, situacao: line.situacao})

CREATE INDEX ON :Medico(username)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/obi10/trabalhoDados/master/jupyter/data/Recomendacao.csv' AS line
MATCH (m:Medico {username: line.usernameMedico})
MATCH (p:Paciente {id: line.idPaciente})
CREATE (m)-[:Tratamento {id: line.idTratamento}]->(p)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/obi10/trabalhoDados/master/jupyter/data/Tratamento.csv' AS line
MATCH (m)-[t:Tratamento {id: line.id}]->(p)
SET t.idRemedio = line.idRemedio
SET t.apreDosagem = line.apreDosagem
SET t.dosis_mcg = line.dosis_mcg
SET t.dosis_mg = line.dosis_mg
SET t.dosis_ml = line.dosis_ml
SET t.numGotas = line.numGotas
SET t.repeticao = line.repeticao
SET t.numVezesDia = line.numVezesDia
SET t.numDias = line.numDias

MATCH (m)-[t:Tratamento]->(p)
MATCH (p)-[:historico]->(d)
MATCH (p1: Paciente)
MATCH (s: Sintoma)
RETURN m, p, d, p1, s